In [1]:
import pandas as pd
# Dependencies
import requests
import json


# Find all Station Location (based in sta 2020)

In [2]:
#202012-capitalbikeshare-tripdata.csv latest record has to be used
df1=pd.read_csv("data/2020/202012-capitalbikeshare-tripdata.csv")
df1.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C04E20007D039277,docked_bike,2020-12-02 09:10:36,2020-12-02 09:24:12,18th St & Wyoming Ave NW,31114.0,18th St & Pennsylvania Ave NW,31242.0,38.918809,-77.041571,38.899680,-77.041539,casual
1,A488C0D9F4761D06,classic_bike,2020-12-31 12:46:29,2020-12-31 14:01:07,Potomac & Pennsylvania Ave SE,31606.0,17th St & Independence Ave SW,31290.0,38.880300,-76.986200,38.888097,-77.038325,casual
2,9E7E97A927A85552,classic_bike,2020-12-31 12:47:03,2020-12-31 14:01:04,Potomac & Pennsylvania Ave SE,31606.0,17th St & Independence Ave SW,31290.0,38.880300,-76.986200,38.888097,-77.038325,casual
3,565C5C331ABAD77B,classic_bike,2020-12-29 13:50:51,2020-12-29 14:12:22,Wakefield High School,31054.0,S Glebe Rd & Potomac Ave,31010.0,38.847129,-77.112550,38.842600,-77.050200,member
4,7576B3F4B4D6ADFF,classic_bike,2020-12-27 12:30:28,2020-12-27 12:36:12,Washington-Liberty High School / N Stafford St...,31920.0,Virginia Square Metro / Monroe St & 9th St N,31024.0,38.888282,-77.111066,38.882788,-77.103148,member


In [3]:
#Drop duplicates and create new df with only important columns
StaLoc_df = df1.drop_duplicates(subset=['start_station_id'], keep='first').reset_index(drop=True)
StaLoc=StaLoc_df[['start_station_id', 'start_lat','start_lng','start_station_name']]
StaLoc.head()

,start_station_id,start_lat,start_lng,start_station_name
0,31114.0,38.918809,-77.041571,18th St & Wyoming Ave NW
1,31606.0,38.880300,-76.986200,Potomac & Pennsylvania Ave SE
2,31054.0,38.847129,-77.112550,Wakefield High School
3,31920.0,38.888282,-77.111066,Washington-Liberty High School / N Stafford St...
4,31269.0,38.887378,-77.001955,3rd St & Pennsylvania Ave SE


In [4]:
#Rename and sort station id column
StaLoc = StaLoc.rename(columns={'start_station_id': 'station_id', 
                                  'start_lat': 'lat',
                                  'start_lng': 'lng',
                                  'start_station_name':'address'})
StaLoc=StaLoc.sort_values(by=['station_id']).reset_index(drop=True)
#Drops rows with station id 0
StaLoc=StaLoc[StaLoc.station_id!=0].dropna()
StaLoc.head()                

,station_id,lat,lng,address
0,31000.0,38.858971,-77.053230,Eads St & 15th St S
1,31001.0,38.857250,-77.053320,18th St & S Eads St
2,31002.0,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003.0,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004.0,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [5]:
#df dimension
StaLoc.shape

(610, 4)

In [6]:
#Data type
StaLoc['station_id']=StaLoc['station_id'].astype('int64')
StaLoc.dtypes

station_id      int64
lat           float64
lng           float64
address        object
dtype: object

# 2019 find total trips by station

In [7]:
#Load all files for 2019
df1=pd.read_csv("data/2019/201901-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2019/201902-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2019/201903-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2019/201904-capitalbikeshare-tripdata.csv")
df5=pd.read_csv("data/2019/201905-capitalbikeshare-tripdata.csv")
df6=pd.read_csv("data/2019/201906-capitalbikeshare-tripdata.csv")
df7=pd.read_csv("data/2019/201907-capitalbikeshare-tripdata.csv")
df8=pd.read_csv("data/2019/201908-capitalbikeshare-tripdata.csv")
df9=pd.read_csv("data/2019/201909-capitalbikeshare-tripdata.csv")
df10=pd.read_csv("data/2019/201910-capitalbikeshare-tripdata.csv")
df11=pd.read_csv("data/2019/201911-capitalbikeshare-tripdata.csv")
df12=pd.read_csv("data/2019/201912-capitalbikeshare-tripdata.csv")

In [8]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df5.shape, df6.shape,df7.shape, df8.shape,df9.shape, df10.shape,df11.shape, df12.shape,

((150780, 9),
 (158130, 9),
 (253811, 9),
 (337704, 9),
 (350144, 9),
 (356645, 9),
 (360044, 9),
 (360225, 9),
 (337552, 9),
 (223512, 9),
 (161878, 9))

In [9]:
#Append all columns 
dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (3398417, 9))

In [10]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2019']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2019
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,31203
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,31321
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,31104
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,31281
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,31014


In [11]:
test_2019_df=df.groupby(['station_id', 'member_type']).agg({'trips_2019':'count'}).reset_index()
test_2019_df.head()

,station_id,member_type,trips_2019
0,0,Casual,20
1,0,Member,478
2,31000,Casual,341
3,31000,Member,1222
4,31001,Casual,298


In [12]:
#create df with station_id and trips for that specifif station
CBS_2019_df=df.groupby(['station_id']).agg({'trips_2019':'count'}).reset_index()
CBS_2019_df.head()

,station_id,trips_2019
0,0,498
1,31000,1563
2,31001,1993
3,31002,5451
4,31003,4338


In [13]:
#drop rows with 0 value in station_id
CBS_2019_df=CBS_2019_df[CBS_2019_df.station_id!=0].dropna()
CBS_2019_df.head()

,station_id,trips_2019
1,31000,1563
2,31001,1993
3,31002,5451
4,31003,4338
5,31004,2127


In [14]:
#shape
CBS_2019_df.shape

(581, 2)

In [15]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2019_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2019,lat,lng,address
0,31000,1563,38.858971,-77.053230,Eads St & 15th St S
1,31001,1993,38.857250,-77.053320,18th St & S Eads St
2,31002,5451,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,4338,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,2127,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [16]:
#shape
CBS_2019_df.shape

(581, 2)

# 2018 find total trips by station

In [17]:
#Load all files for 2018
df1=pd.read_csv("data/2018/201801-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2018/201802-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2018/201803-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2018/201804-capitalbikeshare-tripdata.csv")
df5=pd.read_csv("data/2018/201805-capitalbikeshare-tripdata.csv")
df6=pd.read_csv("data/2018/201806-capitalbikeshare-tripdata.csv")
df7=pd.read_csv("data/2018/201807-capitalbikeshare-tripdata.csv")
df8=pd.read_csv("data/2018/201808-capitalbikeshare-tripdata.csv")
df9=pd.read_csv("data/2018/201809-capitalbikeshare-tripdata.csv")
df10=pd.read_csv("data/2018/201810-capitalbikeshare-tripdata.csv")
df11=pd.read_csv("data/2018/201811-capitalbikeshare-tripdata.csv")
df12=pd.read_csv("data/2018/201812-capitalbikeshare-tripdata.csv")

In [18]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df5.shape, df6.shape,df7.shape, df8.shape,df9.shape, df10.shape,df11.shape, df12.shape,

((168590, 9),
 (182378, 9),
 (238998, 9),
 (374115, 9),
 (392338, 9),
 (404761, 9),
 (403866, 9),
 (325800, 9),
 (343021, 9),
 (221057, 9),
 (158853, 9))

In [19]:
#Append all columns 
dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (3542684, 9))

In [20]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2018']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2018
0,552,2018-01-01 00:05:06,2018-01-01 00:14:18,31104,Adams Mill & Columbia Rd NW,31400,Georgia & New Hampshire Ave NW,W00886,Member,31104
1,1282,2018-01-01 00:14:30,2018-01-01 00:35:53,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W01435,Casual,31321
2,1265,2018-01-01 00:14:53,2018-01-01 00:35:58,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W21242,Casual,31321
3,578,2018-01-01 00:15:31,2018-01-01 00:25:09,31406,14th & Upshur St NW,31103,16th & Harvard St NW,W21322,Casual,31406
4,372,2018-01-01 00:18:02,2018-01-01 00:24:15,31618,4th & East Capitol St NE,31619,Lincoln Park / 13th & East Capitol St NE,W00119,Member,31618


In [21]:
#create df with station_id and trips for that specifif station
CBS_2018_df=df.groupby(['station_id']).agg({'trips_2018':'count'}).reset_index()
CBS_2018_df.head()

,station_id,trips_2018
0,31000,2472
1,31001,2441
2,31002,6469
3,31003,4794
4,31004,2614


In [22]:
#drop rows with 0 value in station_id
CBS_2018_df=CBS_2018_df[CBS_2018_df.station_id!=0].dropna()
CBS_2018_df.head()

,station_id,trips_2018
0,31000,2472
1,31001,2441
2,31002,6469
3,31003,4794
4,31004,2614


In [23]:
#shape
CBS_2018_df.shape

(528, 2)

In [24]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2018_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2018,lat,lng,address
0,31000,2472,38.858971,-77.053230,Eads St & 15th St S
1,31001,2441,38.857250,-77.053320,18th St & S Eads St
2,31002,6469,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,4794,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,2614,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [25]:
#shape
CBS_2018_df.shape

(528, 2)

# 2017 find total trips by station

In [26]:
#Load all files for 2017
df1=pd.read_csv("data/2017/2017Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2017/2017Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2017/2017Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2017/2017Q4-capitalbikeshare-tripdata.csv")

In [27]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((646510, 9), (1104418, 9), (1191585, 9), (815264, 9))

In [28]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (3757777, 9))

In [29]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2017']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2017
0,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member,31634
1,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual,31258
2,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual,31289
3,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual,31289
4,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual,31258


In [30]:
#create df with station_id and trips for that specifif station
CBS_2017_df=df.groupby(['station_id']).agg({'trips_2017':'count'}).reset_index()
CBS_2017_df.head()

,station_id,trips_2017
0,31000,2529
1,31001,2428
2,31002,7060
3,31003,5516
4,31004,2770


In [31]:
CBS_2017_df=CBS_2017_df[CBS_2017_df.station_id!=0].dropna()
CBS_2017_df.head()

,station_id,trips_2017
0,31000,2529
1,31001,2428
2,31002,7060
3,31003,5516
4,31004,2770


In [32]:
#shape
CBS_2017_df.shape

(487, 2)

In [33]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2017_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2017,lat,lng,address
0,31000,2529,38.858971,-77.053230,Eads St & 15th St S
1,31001,2428,38.857250,-77.053320,18th St & S Eads St
2,31002,7060,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,5516,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,2770,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [34]:
#shape
CBS_2017_df.shape

(487, 2)

# 2016 find total trips by station

In [35]:
#Load all files for 2016
df1=pd.read_csv("data/2016/2016Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2016/2016Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2016/2016Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2016/2016Q4-capitalbikeshare-tripdata.csv")

In [36]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((552399, 9), (942333, 9), (1068194, 9), (771068, 9))

In [37]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (3333994, 9))

In [38]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2016']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2016
0,166,2016-01-01 00:06:58,2016-01-01 00:09:44,31102,11th & Kenyon St NW,31105,14th & Harvard St NW,W01346,Member,31102
1,448,2016-01-01 00:10:20,2016-01-01 00:17:48,32039,Old Georgetown Rd & Southwick St,32002,Bethesda Ave & Arlington Rd,W22202,Member,32039
2,715,2016-01-01 00:13:52,2016-01-01 00:25:48,31222,New York Ave & 15th St NW,31214,17th & Corcoran St NW,W21427,Member,31222
3,213,2016-01-01 00:15:29,2016-01-01 00:19:03,31506,1st & Rhode Island Ave NW,31509,New Jersey Ave & R St NW,W01294,Member,31506
4,872,2016-01-01 00:16:16,2016-01-01 00:30:49,31041,Prince St & Union St,31048,King St Metro South,W22058,Member,31041


In [39]:
#create df with station_id and trips for that specifif station
CBS_2016_df=df.groupby(['station_id']).agg({'trips_2016':'count'}).reset_index()
CBS_2016_df.head()

,station_id,trips_2016
0,31000,1883
1,31001,2233
2,31002,6208
3,31003,5304
4,31004,3222


In [40]:
CBS_2016_df=CBS_2016_df[CBS_2016_df.station_id!=0].dropna()
CBS_2016_df.head()

,station_id,trips_2016
0,31000,1883
1,31001,2233
2,31002,6208
3,31003,5304
4,31004,3222


In [41]:
#shape
CBS_2016_df.shape

(435, 2)

In [42]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2016_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2016,lat,lng,address
0,31000,1883,38.858971,-77.053230,Eads St & 15th St S
1,31001,2233,38.857250,-77.053320,18th St & S Eads St
2,31002,6208,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,5304,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,3222,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [43]:
#shape
CBS_2016_df.shape

(435, 2)

# 2015 find total trips by station

In [44]:
#Load all files for 2015
df1=pd.read_csv("data/2015/2015Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2015/2015Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2015/2015Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2015/2015Q4-capitalbikeshare-tripdata.csv")

In [45]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((423719, 9), (999818, 9), (1056366, 9), (706003, 9))

In [46]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (3185906, 9))

In [47]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2015']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2015
0,2389,2015-01-01 00:02:44,2015-01-01 00:42:33,31271,Constitution Ave & 2nd St NW/DOL,31254,15th & K St NW,W01140,Casual,31271
1,2394,2015-01-01 00:02:46,2015-01-01 00:42:41,31271,Constitution Ave & 2nd St NW/DOL,31254,15th & K St NW,W00612,Casual,31271
2,468,2015-01-01 00:04:32,2015-01-01 00:12:20,31204,20th & E St NW,31234,20th & O St NW / Dupont South,W01226,Member,31204
3,348,2015-01-01 00:07:18,2015-01-01 00:13:06,31602,Park Rd & Holmead Pl NW,31117,15th & Euclid St NW,W20216,Member,31602
4,980,2015-01-01 00:09:39,2015-01-01 00:26:00,31247,Jefferson Dr & 14th St SW,31241,Thomas Circle,W21005,Casual,31247


In [48]:
#create df with station_id and trips for that specifif station
CBS_2015_df=df.groupby(['station_id']).agg({'trips_2015':'count'}).reset_index()
CBS_2015_df.head()

,station_id,trips_2015
0,31000,1891
1,31001,2768
2,31002,5127
3,31003,4937
4,31004,2837


In [49]:
CBS_2015_df=CBS_2015_df[CBS_2015_df.station_id!=0].dropna()
CBS_2015_df.head()

,station_id,trips_2015
0,31000,1891
1,31001,2768
2,31002,5127
3,31003,4937
4,31004,2837


In [50]:
#shape
CBS_2015_df.shape

(357, 2)

In [51]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2015_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2015,lat,lng,address
0,31000,1891,38.858971,-77.053230,Eads St & 15th St S
1,31001,2768,38.857250,-77.053320,18th St & S Eads St
2,31002,5127,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,4937,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,2837,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [52]:
#shape
CBS_2015_df.shape

(357, 2)

# 2014 find total trips by station

In [53]:
#Load all files for 2014
df1=pd.read_csv("data/2014/2014Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2014/2014Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2014/2014Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2014/2014Q4-capitalbikeshare-tripdata.csv")

In [54]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((401122, 9), (892819, 9), (972639, 9), (646570, 9))

In [55]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (2913150, 9))

In [56]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2014']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2014
0,1716,2014-01-01 00:06:03,2014-01-01 00:34:40,31102,11th & Kenyon St NW,31632,15th & F St NE,W01363,Member,31102
1,154,2014-01-01 00:11:09,2014-01-01 00:13:43,31602,Park Rd & Holmead Pl NW,31107,Lamont & Mt Pleasant NW,W01433,Member,31602
2,318,2014-01-01 00:12:19,2014-01-01 00:17:38,31223,Convention Center / 7th & M St NW,31509,New Jersey Ave & R St NW,W00065,Member,31223
3,480,2014-01-01 00:14:02,2014-01-01 00:22:02,31203,14th & Rhode Island Ave NW,31109,7th & T St NW,W20903,Member,31203
4,345,2014-01-01 00:15:33,2014-01-01 00:21:18,31241,Thomas Circle,31600,5th & K St NW,W20888,Member,31241


In [57]:
#create df with station_id and trips for that specifif station
CBS_2014_df=df.groupby(['station_id']).agg({'trips_2014':'count'}).reset_index()
CBS_2014_df.head()

,station_id,trips_2014
0,31000,1247
1,31001,2180
2,31002,4672
3,31003,4218
4,31004,2536


In [58]:
CBS_2014_df=CBS_2014_df[CBS_2014_df.station_id!=0].dropna()
CBS_2014_df.head()

,station_id,trips_2014
0,31000,1247
1,31001,2180
2,31002,4672
3,31003,4218
4,31004,2536


In [59]:
#shape
CBS_2014_df.shape

(346, 2)

In [60]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2014_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2014,lat,lng,address
0,31000,1247,38.858971,-77.053230,Eads St & 15th St S
1,31001,2180,38.857250,-77.053320,18th St & S Eads St
2,31002,4672,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,4218,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,2536,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [61]:
#shape
CBS_2014_df.shape

(346, 2)

# 2013 find total trips by station

In [62]:
#Load all files for 2013
df1=pd.read_csv("data/2013/2013Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2013/2013Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2013/2013Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2013/2013Q4-capitalbikeshare-tripdata.csv")

In [63]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((392457, 9), (740672, 9), (838746, 9), (583666, 9))

In [64]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (2555541, 9))

In [65]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2013']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2013
0,688,2013-01-01 00:03:55,2013-01-01 00:15:24,31101,14th & V St NW,31106,Calvert & Biltmore St NW,W01210,Member,31101
1,699,2013-01-01 00:04:39,2013-01-01 00:16:19,31236,37th & O St NW / Georgetown University,31304,36th & Calvert St NW / Glover Park,W01075,Member,31236
2,365,2013-01-01 00:10:01,2013-01-01 00:16:06,31257,22nd & I St NW / Foggy Bottom,31239,Rhode Island & Connecticut Ave NW,w00527,Member,31257
3,292,2013-01-01 00:12:55,2013-01-01 00:17:47,31614,11th & H St NE,31612,D St & Maryland Ave NE,W00040,Member,31614
4,222,2013-01-01 00:13:24,2013-01-01 00:17:07,31239,Rhode Island & Connecticut Ave NW,31214,17th & Corcoran St NW,W20891,Member,31239


In [66]:
#create df with station_id and trips for that specifif station
CBS_2013_df=df.groupby(['station_id']).agg({'trips_2013':'count'}).reset_index()
CBS_2013_df.head()

,station_id,trips_2013
0,31000,1128
1,31001,2345
2,31002,4625
3,31003,2892
4,31004,1626


In [67]:
CBS_2013_df=CBS_2013_df[CBS_2013_df.station_id!=0].dropna()
CBS_2013_df.head()

,station_id,trips_2013
0,31000,1128
1,31001,2345
2,31002,4625
3,31003,2892
4,31004,1626


In [68]:
#shape
CBS_2013_df.shape

(305, 2)

In [69]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2013_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2013,lat,lng,address
0,31000,1128,38.858971,-77.053230,Eads St & 15th St S
1,31001,2345,38.857250,-77.053320,18th St & S Eads St
2,31002,4625,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,2892,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,1626,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [70]:
#shape
CBS_2013_df.shape

(305, 2)

# 2012 find total trips by station

In [71]:
#Load all files for 2012
df1=pd.read_csv("data/2012/2012Q1-capitalbikeshare-tripdata.csv")
df2=pd.read_csv("data/2012/2012Q2-capitalbikeshare-tripdata.csv")
df3=pd.read_csv("data/2012/2012Q3-capitalbikeshare-tripdata.csv")
df4=pd.read_csv("data/2012/2012Q4-capitalbikeshare-tripdata.csv")

In [72]:
#Validate all files has the same columns
df1.shape, df2.shape,df3.shape,df4.shape

((361317, 9), (566832, 9), (629917, 9), (470845, 9))

In [73]:
#Append all columns 
dfs=[df1,df2,df3,df4]
df=pd.concat(dfs)
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (2028911, 9))

In [74]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2012']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2012
0,475,2012-01-01 00:04:00,2012-01-01 00:11:56,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member,31245
1,1162,2012-01-01 00:10:05,2012-01-01 00:29:28,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual,31400
2,1145,2012-01-01 00:10:23,2012-01-01 00:29:28,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member,31400
3,485,2012-01-01 00:15:41,2012-01-01 00:23:46,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member,31101
4,471,2012-01-01 00:15:42,2012-01-01 00:23:34,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member,31102


In [75]:
#create df with station_id and trips for that specifif station
CBS_2012_df=df.groupby(['station_id']).agg({'trips_2012':'count'}).reset_index()
CBS_2012_df.head()

,station_id,trips_2012
0,31000,1353
1,31001,3393
2,31002,4210
3,31003,2443
4,31004,1623


In [76]:
CBS_2012_df=CBS_2012_df[CBS_2012_df.station_id!=0].dropna()
CBS_2012_df.head()

,station_id,trips_2012
0,31000,1353
1,31001,3393
2,31002,4210
3,31003,2443
4,31004,1623


In [77]:
#shape
CBS_2012_df.shape

(191, 2)

In [78]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2012_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2012,lat,lng,address
0,31000,1353,38.858971,-77.053230,Eads St & 15th St S
1,31001,3393,38.857250,-77.053320,18th St & S Eads St
2,31002,4210,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,2443,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,1623,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [79]:
#shape
CBS_2012_df.shape

(191, 2)

# 2011 find total trips by station

In [80]:
#Load all files for 2011
df=pd.read_csv("data/2011/2011-capitalbikeshare-tripdata.csv")

In [81]:
#columns and shape
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (1226767, 9))

In [82]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2011']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2011
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,31620
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,31105
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,31400
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,31111
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,31104


In [83]:
#create df with station_id and trips for that specifif station
CBS_2011_df=df.groupby(['station_id']).agg({'trips_2011':'count'}).reset_index()
CBS_2011_df.head()

,station_id,trips_2011
0,31000,1171
1,31001,3492
2,31002,2883
3,31003,3432
4,31004,1598


In [84]:
CBS_2011_df=CBS_2011_df[CBS_2011_df.station_id!=0].dropna()
CBS_2011_df.head()

,station_id,trips_2011
0,31000,1171
1,31001,3492
2,31002,2883
3,31003,3432
4,31004,1598


In [85]:
#shape
CBS_2011_df.shape

(144, 2)

In [86]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2011_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2011,lat,lng,address
0,31000,1171,38.858971,-77.053230,Eads St & 15th St S
1,31001,3492,38.857250,-77.053320,18th St & S Eads St
2,31002,2883,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,3432,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,1598,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [87]:
#shape
CBS_2011_df.shape

(144, 2)

# 2010 find total trips by station

In [88]:
#Load all files for 2010
df=pd.read_csv("data/2010/2010-capitalbikeshare-tripdata.csv")

In [89]:
#columns and shape
df.columns, df.shape

(Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 (115597, 9))

In [90]:
#rename columns
df = df.rename(columns={'Start date': 'start_date', 
                        'End date': 'end_date',
                        'Start station number': 'station_id',
                        'Start station':'start_station',
                        'End station number':'end_station_number',
                        'End station':'end_station',
                        'Bike number':'bike_id',
                        'Member type':'member_type'})
df['trips_2010']=df['station_id']
df.head()

,Duration,start_date,end_date,station_id,start_station,end_station_number,end_station,bike_id,member_type,trips_2010
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,M St & New Jersey Ave SE,31108,4th & M St SW,W00742,Member,31208
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,1st & N St SE,31209,1st & N St SE,W00032,Member,31209
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,5th & K St NW,31100,19th St & Pennsylvania Ave NW,W00993,Member,31600
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,5th & K St NW,31602,Park Rd & Holmead Pl NW,W00344,Member,31600
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,19th St & Pennsylvania Ave NW,31201,15th & P St NW,W00883,Member,31100


In [91]:
#create df with station_id and trips for that specifif station
CBS_2010_df=df.groupby(['station_id']).agg({'trips_2010':'count'}).reset_index()
CBS_2010_df.head()

,station_id,trips_2010
0,31000,185
1,31001,372
2,31002,362
3,31003,463
4,31004,234


In [92]:
CBS_2010_df=CBS_2010_df[CBS_2010_df.station_id!=0].dropna()
CBS_2010_df.head()

,station_id,trips_2010
0,31000,185
1,31001,372
2,31002,362
3,31003,463
4,31004,234


In [93]:
#shape
CBS_2010_df.shape

(106, 2)

In [94]:
#merge with StaLoc to have lat, lng columns
df_test = pd.merge(CBS_2010_df, StaLoc, on=['station_id','station_id'])
df_test.head()

,station_id,trips_2010,lat,lng,address
0,31000,185,38.858971,-77.053230,Eads St & 15th St S
1,31001,372,38.857250,-77.053320,18th St & S Eads St
2,31002,362,38.856425,-77.049232,Crystal Dr & 20th St S
3,31003,463,38.860170,-77.049593,Crystal Dr & 15th St S
4,31004,234,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St


In [95]:
#shape
CBS_2010_df.shape

(106, 2)

# Totals by year

In [96]:
#Merge all CBS_XXXX-df with StaLoc df
CBS_totals_df = pd.merge(StaLoc, CBS_2010_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2011_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2012_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2013_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2014_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2015_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2016_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2017_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2018_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df = pd.merge(CBS_totals_df, CBS_2019_df, how='left', left_on='station_id', right_on='station_id')
CBS_totals_df=CBS_totals_df.fillna(0)
CBS_totals_df.head()

,station_id,lat,lng,address,trips_2010,trips_2011,trips_2012,trips_2013,trips_2014,trips_2015,trips_2016,trips_2017,trips_2018,trips_2019
0,31000,38.858971,-77.053230,Eads St & 15th St S,185.0,1171.0,1353.0,1128.0,1247.0,1891.0,1883.0,2529.0,2472.0,1563.0
1,31001,38.857250,-77.053320,18th St & S Eads St,372.0,3492.0,3393.0,2345.0,2180.0,2768.0,2233.0,2428.0,2441.0,1993.0
2,31002,38.856425,-77.049232,Crystal Dr & 20th St S,362.0,2883.0,4210.0,4625.0,4672.0,5127.0,6208.0,7060.0,6469.0,5451.0
3,31003,38.860170,-77.049593,Crystal Dr & 15th St S,463.0,3432.0,2443.0,2892.0,4218.0,4937.0,5304.0,5516.0,4794.0,4338.0
4,31004,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St,234.0,1598.0,1623.0,1626.0,2536.0,2837.0,3222.0,2770.0,2614.0,2127.0


In [97]:
#get shape
CBS_totals_df.dtypes, CBS_totals_df.shape

(station_id      int64
 lat           float64
 lng           float64
 address        object
 trips_2010    float64
 trips_2011    float64
 trips_2012    float64
 trips_2013    float64
 trips_2014    float64
 trips_2015    float64
 trips_2016    float64
 trips_2017    float64
 trips_2018    float64
 trips_2019    float64
 dtype: object,
 (610, 14))

In [98]:
CBS_totals_df.tail()

,station_id,lat,lng,address,trips_2010,trips_2011,trips_2012,trips_2013,trips_2014,trips_2015,trips_2016,trips_2017,trips_2018,trips_2019
605,32606,38.878085,-77.153953,N Roosevelt St & Roosevelt Blvd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,662.0
606,32607,38.879773,-77.178394,S Maple Ave & S Washington St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0
607,32608,38.885379,-77.173522,Falls Church City Hall / Park Ave & Little Fal...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,353.0
608,32609,38.885621,-77.166917,W Columbia St & N Washington St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,420.0
609,32901,38.963810,-77.010266,6035 Warehouse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
convert_dict = {'trips_2010': int,
                'trips_2011': int,
                'trips_2012': int,
                'trips_2013': int,
                'trips_2014': int,
                'trips_2015': int,
                'trips_2016': int,
                'trips_2017': int,
                'trips_2018': int,
                'trips_2019': int,
               }
  
CBS_totals_df = CBS_totals_df.astype(convert_dict)
CBS_totals_df.dtypes

station_id      int64
lat           float64
lng           float64
address        object
trips_2010      int32
trips_2011      int32
trips_2012      int32
trips_2013      int32
trips_2014      int32
trips_2015      int32
trips_2016      int32
trips_2017      int32
trips_2018      int32
trips_2019      int32
dtype: object

In [100]:
CBS_totals_df.head()

,station_id,lat,lng,address,trips_2010,trips_2011,trips_2012,trips_2013,trips_2014,trips_2015,trips_2016,trips_2017,trips_2018,trips_2019
0,31000,38.858971,-77.053230,Eads St & 15th St S,185,1171,1353,1128,1247,1891,1883,2529,2472,1563
1,31001,38.857250,-77.053320,18th St & S Eads St,372,3492,3393,2345,2180,2768,2233,2428,2441,1993
2,31002,38.856425,-77.049232,Crystal Dr & 20th St S,362,2883,4210,4625,4672,5127,6208,7060,6469,5451
3,31003,38.860170,-77.049593,Crystal Dr & 15th St S,463,3432,2443,2892,4218,4937,5304,5516,4794,4338
4,31004,38.857880,-77.059505,Aurora Hills Cmty Ctr / 18th St & S Hayes St,234,1598,1623,1626,2536,2837,3222,2770,2614,2127


In [102]:
# Export file
CBS_totals_df.to_csv("data/CBS_totals.csv")